In [66]:
import pandas as pd

In [67]:
df = pd.read_csv('./output.tsv', sep="\t")

In [68]:
df.head()

,date,description,type,amount,source,categories
0,18/06/2023,Porção janta,OUTCOME,"-R$82,99",ALELO,ALIMENTAÇÃO
1,18/06/2023,Compensação porção,INCOME,"R$75,00",NaN,COMPENSAÇÃO
2,18/06/2023,Compras Supermercado,OUTCOME,"-R$90,39",ALELO,ALIMENTAÇÃO
3,18/06/2023,Pratos,OUTCOME,"-R$59,94",APÊ,NaN
4,18/06/2023,Talheres e filtro de linha,OUTCOME,"-R$129,98",APÊ,NaN


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         406 non-null    object
 1   description  406 non-null    object
 2   type         406 non-null    object
 3    amount      406 non-null    object
 4   source       401 non-null    object
 5   categories   389 non-null    object
dtypes: object(6)
memory usage: 19.2+ KB


In [70]:
df.rename(columns={' amount ': 'amount'}, inplace=True)

In [71]:
from datetime import datetime

df["date"] = df["date"].map(lambda date: datetime.strptime(date, "%d/%m/%Y"))

In [72]:
df["amount"] = df["amount"].map(lambda amt: str(amt).replace('-', '').replace('R$', '').replace('.', '').replace(',', '.')).astype(float)

In [73]:
df.rename(columns={'source': 'tags/0'}, inplace=True)

In [74]:
tags = df["categories"].str.split(", ", expand=True).fillna(pd.NA)
tags.columns = [f'tags/{i+1}' for i in range(tags.shape[1])]
tags

,tags/1,tags/2
0,ALIMENTAÇÃO,NaN
1,COMPENSAÇÃO,NaN
2,ALIMENTAÇÃO,NaN
3,<NA>,NaN
4,<NA>,NaN
...,...,...
401,COMPENSAÇÃO,<NA>
402,DESPESAS FIXAS,<NA>
403,GABY,SAÚDE
404,DESPESAS FIXAS,<NA>


In [75]:
df = pd.concat([df, tags], axis=1).drop(columns=['categories'])

In [76]:
df.head()

,date,description,type,amount,tags/0,tags/1,tags/2
0,2023-06-18,Porção janta,OUTCOME,82.99,ALELO,ALIMENTAÇÃO,<NA>
1,2023-06-18,Compensação porção,INCOME,75.00,NaN,COMPENSAÇÃO,<NA>
2,2023-06-18,Compras Supermercado,OUTCOME,90.39,ALELO,ALIMENTAÇÃO,<NA>
3,2023-06-18,Pratos,OUTCOME,59.94,APÊ,<NA>,<NA>
4,2023-06-18,Talheres e filtro de linha,OUTCOME,129.98,APÊ,<NA>,<NA>


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         406 non-null    datetime64[ns]
 1   description  406 non-null    object        
 2   type         406 non-null    object        
 3   amount       406 non-null    float64       
 4   tags/0       401 non-null    object        
 5   tags/1       389 non-null    object        
 6   tags/2       39 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 22.3+ KB


In [78]:
df.to_csv("./input.tsv", sep="\t", lineterminator='\r\n', na_rep="", index=False)